In [72]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import pycountry
import pypopulation
from geojson import FeatureCollection, dump
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from matplotlib.pyplot import figure

## Geocoding
Assigning countries coordinates

In [73]:
load_dotenv()
TOKEN=os.getenv("MAPBOX_TOKEN")
df = pd.read_csv("../output/processed/flows.csv") # ../output/processed/
origins = list(df.origin.unique())
destinations = list(df.destination.unique())

In [74]:
in_first = set(origins)
in_second = set(destinations)
in_second_but_not_in_first = in_second - in_first
countries = origins + list(in_second_but_not_in_first)

In [75]:
df = pd.DataFrame({"country":countries})

In [76]:
headers = {'Accept': 'application/json'}
lst = []
for i in countries:
    obj = {}
    obj["type"]="Feature"
    obj["properties"]={"name": pycountry.countries.get(alpha_2=i).name}
    pop = pypopulation.get_population_a2(i)
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{i}.json?&types=country&access_token={TOKEN}"
    r = requests.get(url)
    jason = r.json()
    obj["properties"]["code"] = i.lower()
    coords = jason["features"][0]["center"]
    obj["geometry"] = {"type":"Point", "coordinates": coords}
    lst.append(obj)
    df.loc[df["country"] == i, "lat"] = coords[0]
    df.loc[df["country"] == i, "lon"] = coords[1]
    df.loc[df["country"] == i, "name"] = obj["properties"]["name"]
    df.loc[df["country"] == i, "population"] = pop

In [77]:
with open('../output/processed/countries.geojson', 'w') as f:
    dump(FeatureCollection(lst), f)

In [78]:
df['population'] = df['population'].fillna(0).astype(int)

In [79]:
# filling one country manually
df.loc[df[df["name"]=="Saint Barthélemy"].index, 'population'] = 10457
df = df.astype({'population':'int'})

## Clustering


In [80]:
X = df[["lat","lon"]]

In [81]:
lst = []
s = len(X)
# 6 zoom levels
for i in range(5):
    s -= 23
    lst.append(s)
lst

[132, 109, 86, 63, 40]

In [82]:
for count,i in enumerate(lst):
    hierarchical_cluster = AgglomerativeClustering(n_clusters=i, metric='euclidean', linkage='ward')
    labels = hierarchical_cluster.fit_predict(X)
    clf = NearestCentroid()
    clf.fit(X, labels)
    df[f"label_{count+1}"] = labels
    df[f"lat_{count+1}"] = df.apply(lambda x: clf.centroids_[x[f"label_{count+1}"]][0], axis=1)
    df[f"lon_{count+1}"] = df.apply(lambda x: clf.centroids_[x[f"label_{count+1}"]][1], axis=1)


In [83]:
df.rename(columns={"lon": "lon_0", "lat": "lat_0"}, inplace=True)
df

,country,lat_0,lon_0,name,population,label_1,lat_1,lon_1,label_2,lat_2,lon_2,label_3,lat_3,lon_3,label_4,lat_4,lon_4,label_5,lat_5,lon_5
0,ad,1.576766,42.548654,Andorra,77142,97,1.576766,42.548654,97,1.576766,42.548654,7,2.097777,45.186779,15,2.097777,45.186779,13,-2.105716,42.817352
1,ae,10.525771,1.612130,United Arab Emirates,9770529,67,10.525771,1.612130,67,10.525771,1.612130,67,10.525771,1.612130,3,8.912128,6.933350,15,8.912128,6.933350
2,af,66.026471,33.838806,Afghanistan,38041754,93,66.026471,33.838806,93,66.026471,33.838806,14,66.860854,31.012595,30,66.860854,31.012595,4,70.732796,28.074786
3,al,2.632388,28.163240,Albania,2854191,9,2.632388,28.163240,20,2.632388,28.163240,41,2.632388,28.163240,41,2.632388,28.163240,1,7.762685,27.790147
4,am,-97.922211,39.381266,Armenia,2957731,15,-97.922211,39.381266,32,-97.922211,39.381266,32,-97.922211,39.381266,32,-97.922211,39.381266,32,-97.922211,39.381266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,tj,71.042004,38.838508,Tajikistan,9321018,21,71.042004,38.838508,21,71.042004,38.838508,34,72.798800,40.151781,34,72.798800,40.151781,14,67.090625,41.867274
151,ug,32.386218,1.279964,Uganda,44269594,19,32.386218,1.279964,39,32.386218,1.279964,9,35.122050,0.904913,40,35.122050,0.904913,8,31.137000,0.212303
152,hk,114.162550,22.279356,Hong Kong,7507400,36,114.162550,22.279356,36,114.162550,22.279356,74,114.162550,22.279356,2,117.572165,18.433072,0,111.125326,16.577226
153,sa,44.652426,23.384784,Saudi Arabia,34268528,28,44.652426,23.384784,28,44.652426,23.384784,57,44.652426,23.384784,12,46.137178,19.661310,19,50.846777,23.131173


In [84]:
df

,country,lat_0,lon_0,name,population,label_1,lat_1,lon_1,label_2,lat_2,lon_2,label_3,lat_3,lon_3,label_4,lat_4,lon_4,label_5,lat_5,lon_5
0,ad,1.576766,42.548654,Andorra,77142,97,1.576766,42.548654,97,1.576766,42.548654,7,2.097777,45.186779,15,2.097777,45.186779,13,-2.105716,42.817352
1,ae,10.525771,1.612130,United Arab Emirates,9770529,67,10.525771,1.612130,67,10.525771,1.612130,67,10.525771,1.612130,3,8.912128,6.933350,15,8.912128,6.933350
2,af,66.026471,33.838806,Afghanistan,38041754,93,66.026471,33.838806,93,66.026471,33.838806,14,66.860854,31.012595,30,66.860854,31.012595,4,70.732796,28.074786
3,al,2.632388,28.163240,Albania,2854191,9,2.632388,28.163240,20,2.632388,28.163240,41,2.632388,28.163240,41,2.632388,28.163240,1,7.762685,27.790147
4,am,-97.922211,39.381266,Armenia,2957731,15,-97.922211,39.381266,32,-97.922211,39.381266,32,-97.922211,39.381266,32,-97.922211,39.381266,32,-97.922211,39.381266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,tj,71.042004,38.838508,Tajikistan,9321018,21,71.042004,38.838508,21,71.042004,38.838508,34,72.798800,40.151781,34,72.798800,40.151781,14,67.090625,41.867274
151,ug,32.386218,1.279964,Uganda,44269594,19,32.386218,1.279964,39,32.386218,1.279964,9,35.122050,0.904913,40,35.122050,0.904913,8,31.137000,0.212303
152,hk,114.162550,22.279356,Hong Kong,7507400,36,114.162550,22.279356,36,114.162550,22.279356,74,114.162550,22.279356,2,117.572165,18.433072,0,111.125326,16.577226
153,sa,44.652426,23.384784,Saudi Arabia,34268528,28,44.652426,23.384784,28,44.652426,23.384784,57,44.652426,23.384784,12,46.137178,19.661310,19,50.846777,23.131173


In [86]:
df.to_csv("../output/processed/countries.csv", index=False)